Assignment


1. Consider the two variables: TOTAL_REVENUE and TOTAL_EXPENDITURE. Do these variables have outlier values?

2. If you detect outliers in the TOTAL_REVENUE and TOTAL_EXPENDITURE variables, apply the techniques you learned in this checkpoint to eliminate them and validate that there's no outlier values after you handled them.

3. Create another variable by subtracting the original TOTAL_EXPENDITURE from TOTAL_REVENUE (before you eliminated the outliers). You can think of it as a kind of budget deficit in education. Do you find any outlier values in this new variable? If so, eliminate them using the technique you think most suitable.

4. Now create another variable by subtracting the TOTAL_EXPENDITURE from TOTAL_REVENUE. This time, use the outlier eliminated versions of TOTAL_EXPENDITURE from TOTAL_REVENUE. In this newly created variable, can you find any outliers? If so, eliminate them.

5. Compare some basic descriptive statistics of the budget variables you end up with in the 3rd and the 4th questions. Do you see any differences?

6. If our variable of interest is the budget deficit variable, which method do you think is the appropriate in dealing with the outliers in this variable: the method in the 3rd question or the one in the 4th question?



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

data = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [3]:
data.head()

PRIMARY_KEY       STATE  YEAR  ENROLL  TOTAL_REVENUE  FEDERAL_REVENUE  \
0     1992_ALABAMA     ALABAMA  1992     NaN      2678885.0         304177.0   
1      1992_ALASKA      ALASKA  1992     NaN      1049591.0         106780.0   
2     1992_ARIZONA     ARIZONA  1992     NaN      3258079.0         297888.0   
3    1992_ARKANSAS    ARKANSAS  1992     NaN      1711959.0         178571.0   
4  1992_CALIFORNIA  CALIFORNIA  1992     NaN     26260025.0        2072470.0   

   STATE_REVENUE  LOCAL_REVENUE  TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
0      1659028.0       715680.0          2653798.0                1481703.0   
1       720711.0       222100.0           972488.0                 498362.0   
2      1369815.0      1590376.0          3401580.0                1435908.0   
3       958785.0       574603.0          1743022.0                 964323.0   
4     16546514.0      7641041.0         27138832.0               14358922.0   

   ...  GRADES_4_G  GRADES_8_G  GRADES_12_G  GRADES_1_8_G  GRADES_9_12_G  \
0  ...     57948.0     58025.0      41167.0      471564.0       196386.0   
1  ...      9748.0      8789.0       6714.0       79117.0        30847.0   
2  ...     55433.0     49081.0      37410.0      437127.0       175210.0   
3  ...     34632.0     36011.0      27651.0      281338.0       123113.0   
4  ...    418418.0    363296.0     270675.0     3286034.0      1372011.0   

   GRADES_ALL_G  AVG_MATH_4_SCORE  AVG_MATH_8_SCORE  AVG_READING_4_SCORE  \
0      676174.0        208.327876        252.187522           207.963517   
1      112335.0               NaN               NaN                  NaN   
2      614881.0        215.253932        265.366278           206.212716   
3      405259.0        210.206028        256.312090           208.634458   
4     4717112.0        208.398961        260.892247           196.764414   

   AVG_READING_8_SCORE  
0                  NaN  
1           258.859712  
2           262.169895  
3           264.619665  
4                  NaN  

[5 rows x 25 columns]

1. Consider the two variables: TOTAL_REVENUE and TOTAL_EXPENDITURE. Do these variables have outlier values?

In [4]:
data['TOTAL_REVENUE'] = [float(i) for i in data['TOTAL_REVENUE']]

data['TOTAL_REVENUE'].fillna(0)
zscores = (data['TOTAL_REVENUE'] - data['TOTAL_REVENUE'].mean())/data['TOTAL_REVENUE'].std(ddof=0)
max(zscores)


6.824624816804302

In [5]:
data['TOTAL_EXPENDITURE'] = [float(i) for i in data['TOTAL_EXPENDITURE']]

data['TOTAL_EXPENDITURE'].fillna(0)
zscores = (data['TOTAL_EXPENDITURE'] - data['TOTAL_EXPENDITURE'].mean())/data['TOTAL_EXPENDITURE'].std(ddof=0)
max(zscores)


6.357688973613789

No matter how much we set our threshold, with a standard deviation of at least 6.3 away from the mean, both columns have distant outliers.

2. If you detect outliers in the TOTAL_REVENUE and TOTAL_EXPENDITURE variables, apply the techniques you learned in this checkpoint to eliminate them and validate that there's no outlier values after you handled them.

In [8]:
# Winsorize
data['TOTAL_REVENUE'] = winsorize(data['TOTAL_REVENUE'],(0,0.975))
data['TOTAL_EXPENDITURE'] = winsorize(data['TOTAL_EXPENDITURE'],(0,0.975))


# Z-score check
rev_zscores = (data['TOTAL_REVENUE'] - data['TOTAL_REVENUE'].mean())/data['TOTAL_REVENUE'].std(ddof=0)
exp_zscores = (data['TOTAL_EXPENDITURE'] - data['TOTAL_EXPENDITURE'].mean())/data['TOTAL_EXPENDITURE'].std(ddof=0)

print(max(rev_zscores))
print(min(rev_zscores))
print(max(exp_zscores))
print(min(exp_zscores))

0.13309781989693348
-14.78324970592338
0.13240977488034764
-15.050247983369067


3. Create another variable by subtracting the original TOTAL_EXPENDITURE from TOTAL_REVENUE (before you eliminated the outliers). You can think of it as a kind of budget deficit in education. Do you find any outlier values in this new variable? If so, eliminate them using the technique you think most suitable.

In [6]:
data['INCOME1'] = data['TOTAL_REVENUE'] - data['TOTAL_EXPENDITURE']
zscores = (data['INCOME1'] - data['INCOME1'].mean())/data['INCOME1'].std(ddof=0)
max(zscores)


6.981619878561692

In [7]:
data['INCOME1'] = winsorize(data['INCOME1'],(0,0.975))

4. Now create another variable by subtracting the TOTAL_EXPENDITURE from TOTAL_REVENUE. This time, use the outlier eliminated versions of TOTAL_EXPENDITURE from TOTAL_REVENUE. In this newly created variable, can you find any outliers? If so, eliminate them.

In [9]:
data['INCOME2'] = data['TOTAL_REVENUE'] - data['TOTAL_EXPENDITURE']
zscores = (data['INCOME2'] - data['INCOME2'].mean())/data['INCOME2'].std(ddof=0)
max(zscores)


9.420021595254974

In [14]:
q75, q25 = np.percentile(np.log(data["INCOME2"]), [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((np.log(data["INCOME2"]) > max_val) 
                      | (np.log(data["INCOME2"]) < min_val))[0]))
    ))

The score threshold is: 1.0
Number of outliers is: 0
The score threshold is: 1.5
Number of outliers is: 0
The score threshold is: 2.0
Number of outliers is: 0
The score threshold is: 2.5
Number of outliers is: 0
The score threshold is: 3.0
Number of outliers is: 0
The score threshold is: 3.5
Number of outliers is: 0
The score threshold is: 4.0
Number of outliers is: 0
The score threshold is: 4.5
Number of outliers is: 0


5. Compare some basic descriptive statistics of the budget variables you end up with in the 3rd and the 4th questions. Do you see any differences?

In [11]:
data['INCOME1'].describe()

count    1.492000e+03
mean    -1.451443e+06
std      2.244670e+05
min     -5.487742e+06
25%     -1.426064e+06
50%     -1.426064e+06
75%     -1.426064e+06
max     -1.426064e+06
Name: INCOME1, dtype: float64

In [12]:
data['INCOME2'].describe()

count     1492.000000
mean     13641.402815
std       3751.240444
min     -41458.000000
25%      14069.000000
50%      14069.000000
75%      14069.000000
max      14069.000000
Name: INCOME2, dtype: float64

There is certainly a difference in values. It seems there is a difference especially in the IQR, in addition to a significant difference in standard deviation and mean.

6. If our variable of interest is the budget deficit variable, which method do you think is the appropriate in dealing with the outliers in this variable: the method in the 3rd question or the one in the 4th question?

I would prefer Tukey's method, since the information within the data is preserved and reshaped, rather than being changed through changing data points.